In [ ]:
%matplotlib inline
import menpo.io as mio
from menpo.visualize import print_progress
from menpo.landmark import labeller, face_ibug_68_to_face_ibug_68_trimesh
from menpowidgets import visualize_images
from pathlib import Path

path_to_images = 'C:\\Users\\ch9fod\\Documents\\lfpw\\trainset'
training_images = []
for img in print_progress(mio.import_images(path_to_images, verbose=True)):
    # convert to greyscale
    if img.n_channels == 3:
        img = img.as_greyscale()
    # crop to landmarks bounding box with an extra 20% padding
    img = img.crop_to_landmarks_proportion(0.2)
    # rescale image if its diagonal is bigger than 400 pixels
    d = img.diagonal()
    if d > 400:
        img = img.rescale(400.0 / d)
    # define a TriMesh which will be useful for Piecewise Affine Warp of HolisticAAM
    labeller(img, 'PTS', face_ibug_68_to_face_ibug_68_trimesh)
    # append to list
    training_images.append(img)

In [ ]:
from menpofit.aam import HolisticAAM
from menpo.feature import fast_dsift

# build AAM
aam = HolisticAAM(
    training_images,
    group='face_ibug_68_trimesh',
    verbose=True,
    holistic_features=fast_dsift, 
    diagonal=120, 
    scales=(0.5, 1.0)
)

In [ ]:
from menpofit.aam import LucasKanadeAAMFitter

fitter = LucasKanadeAAMFitter(aam, n_shape=0.9, n_appearance=0.9)

In [ ]:
path_to_lfpw = Path('C:\\Users\\ch9fod\\Documents\\lfpw\\')

# load test and failed images
test_images = []
bboxes = []
for i in mio.import_images(path_to_lfpw / 'MySet', max_images=12, verbose=True):
    # crop image
    i = i.crop_to_landmarks_proportion(0.2)
    # convert it to grayscale if needed
    if i.n_channels == 3:
        i = i.as_greyscale(mode='luminosity')
    # append it to the list
    test_images.append(i)  

In [ ]:
from menpofit.fitter import noisy_shape_from_bounding_box

fitting_results = []

# fit images
for i in test_images:
    # obtain ground truth (original) landmarks
    gt_s = i.landmarks['LJSON'].lms
    
    # generate initialization shape
    initial_s = noisy_shape_from_bounding_box(gt_s, gt_s.bounding_box())
    
    # fit image
    fr = fitter.fit_from_shape(i, initial_s, gt_shape=gt_s)
    fitting_results.append(fr)
    
    # print fitting error
    #print(fr)

In [ ]:
from menpowidgets import visualize_fitting_result

visualize_fitting_result(fitting_results)

In [ ]:
#comments
'''
aam.view_shape_models_widget()
aam.view_appearance_models_widget()
aam.view_aam_widget()

%matplotlib inline
from menpowidgets import visualize_images
visualize_images(training_images)

from menpofit.aam import PatchAAM
from menpo.feature import fast_dsift

patch_aam = PatchAAM(training_images, group='PTS', patch_shape=[(15, 15), (23, 23)],
                     diagonal=150, scales=(0.5, 1.0), holistic_features=fast_dsift,
                     max_shape_components=20, max_appearance_components=150,
                     verbose=True)                   

from menpofit.aam import HolisticAAM
from menpo.feature import fast_dsift

aam = HolisticAAM(training_images, group='face_ibug_68_trimesh', diagonal=150,
                  scales=(0.5, 1.0), holistic_features=fast_dsift, verbose=True,
                  max_shape_components=20, max_appearance_components=150)
                  
fitter = LucasKanadeAAMFitter(aam, lk_algorithm_cls=WibergInverseCompositional,
                              n_shape=[5, 20], n_appearance=[30, 150])                  
                  
result.view(render_initial_shape=True)
result.view_iterations()
result.view_widget()
                  
image.view()   
image = image.crop_to_landmarks_proportion(0.5)

# TEST!!!
#len(bboxes)
#print(test_images[0].landmarks)
#image.landmarks.group_labels[0]
training_images[0].landmarks.group_labels[0]

visualize_images(test_images)

# View
#if len(bboxes) > 0:
#    image.view_landmarks(group='bbox_0', line_colour='red',
#                         render_markers=False, line_width=4);

# Detect
#bboxes = detect(image)
#print("{} detected faces.".format(len(bboxes)))

# View
#if len(bboxes) > 0:
#    image.view_landmarks(group='dlib_0', line_colour='red',
#                         render_markers=False, line_width=4);

test_images[0].view_landmarks(group='bbox_0', line_colour='red',
                     render_markers=False, line_width=4);

#from menpodetect import load_dlib_frontal_face_detector

# Load detector
#detect = load_dlib_frontal_face_detector()

from menpofit.aam import LucasKanadeAAMFitter, WibergInverseCompositional
fitter = LucasKanadeAAMFitter(aam, lk_algorithm_cls=WibergInverseCompositional,
                              n_shape=[5, 20], n_appearance=[30, 150])
                              
image = mio.import_image(path_to_lfpw / 'Jorge1.jpg')
image = image.as_greyscale()

from menpodetect import load_dlib_frontal_face_detector

# Load detector
detect = load_dlib_frontal_face_detector()

#bboxes = detect(image)
bboxes = detect(image, group_prefix='bbox')

print("{} detected faces.".format(len(bboxes)))            

# fiting images
#for i in test_images:
    # fit image
#    fr = fitter.fit_from_bb(i,
#                            bboxes[index],
#                            max_iters=[15, 5],
#                            gt_shape=None)
#    index += 1
#    fitting_results.append(fr)

# print result
#print(result)
                     
'''